In [1]:
import etl_utils as eu
import pandas as pd 
import os

In [2]:
DATAPATH = r"..\data\alternative_data\staging"
PROCESSED_DIR = r"..\\data\\alternative_data\\processed"

In [ ]:
csv_files = eu.get_csv_files(DATAPATH)

In [ ]:
google_trends = "cali-monthly-google_trends.csv"
mobility_report = "us-daily-mobility_report.csv"
consumer_sentiment = "us-monthly-consumer_sentiment.csv"

for file in csv_files:
    file_name = os.path.basename(file)
    data = pd.read_csv(file)
    
    if file_name == google_trends:
        
        data = data.replace("<1", "0")  # replacing unformatted numbers
        
    elif file_name == mobility_report:
        
        date_col = data.pop('date')  
        data.insert(0, 'date', date_col)   # querying file for california data 
        data = data.query("state == 'California'")
        
        file = eu.get_modified_file_path(name, "cali-daily-mobility_report.csv")
    
    elif file_name == consumer_sentiment:
        
        data[["Month", "Year"]] = data[["Month", "Year"]].astype(str) # adjusting date string 
        date_series = data['Year'] + "-" + data['Month']
        data.insert(0, "date", date_series)

    file = eu.change_directory(file, PROCESSED_DIR)
    data.to_csv(file)      

    